# SNP overlap analysis for j5-7 between BrainSeq caudate and GTEx accumbens eQTLs

In [ ]:
library(tidyverse)
library(ggpubr)

In [ ]:
save_ggplots <- function(fn, p, w, h){
    for(ext in c('.pdf', '.png', '.svg')){
        ggsave(paste0(fn, ext), plot=p, width=w, height=h)
    }
}

## Load eQTLs

In [ ]:
cname = c("gene_id", "variant_id", "tss_distance", "ma_samples", "ma_count",
          "maf", "pval_nominal", "slope", "slope_se")
## Junction 5-7
cmd = paste0('zcat /ceph/projects/v4_phase3_paper/analysis/eqtl_analysis/all/junctions/',
             'expression_gct/prepare_expression/fastqtl_nominal/_m/Brainseq_LIBD.allpairs.txt.gz',
             '| grep "chr11:113412884-113415420(-)"')
caudate = data.table::fread(cmd=cmd, header=FALSE, col.names=cname)
dim(caudate)
## Junction 5-7 GTEx accumbens
cmd = paste0('zcat /ceph/projects/v4_phase3_paper/analysis/eqtl_analysis/gtex_accumbens/',
             'junctions/prepare_expression/fastqtl_nominal/_m/GTEx_accumbens.allpairs.txt.gz',
             '| grep "chr11_113412884_113415420"')
gtex_accumbens = data.table::fread(cmd=cmd, header=FALSE, col.names=cname)
dim(gtex_accumbens)
## Junction 5-6 GTEx accumbens
cmd = paste0('zcat /ceph/projects/v4_phase3_paper/analysis/eqtl_analysis/gtex_accumbens/',
             'junctions/prepare_expression/fastqtl_nominal/_m/GTEx_accumbens.allpairs.txt.gz',
             '| grep "chr11_113414462_113415420"')
gtex_accumbens_j56 = data.table::fread(cmd=cmd, header=FALSE, col.names=cname)
dim(gtex_accumbens_j56)

In [ ]:
df = caudate %>% mutate(variant_id=paste0(gsub(':', '_', variant_id), '_b38')) %>%
    inner_join(gtex_accumbens, by=c('variant_id'), suffix=c("_LIBD", "_GTEx"))
#df.to_csv('j57_and_j56_eqtls.tsv', sep='\t')
df %>% head(2)

In [ ]:
title = paste0("eQTLs for DRD2 junctions 5-7 in LIBD caudate and GTEx accumbens\n(n=",dim(df)[1]," SNPs)")

sca = df %>% ggscatter(x="pval_nominal_LIBD", y="pval_nominal_GTEx", alpha=0.4, 
                 ggtheme=theme_pubclean(base_size=20), 
                 xlab="nominal p-value: LIBD caudate",
                 ylab="nominal p-value: GTEx accumbens") +
    yscale("log10", .format=TRUE) + xscale("log10", .format=TRUE) +
    font("xy.title", face="bold") +
    ggtitle(title) + theme(plot.title = element_text(hjust = 0.5, face="bold"))
print(sca)
save_ggplots("eQTL_scatterplot_j57_LIBD_caudate_GTEx_accumbens", sca, 12, 10)

In [ ]:
df2 = gtex_accumbens %>% inner_join(gtex_accumbens_j56, by=c('variant_id'), suffix=c("_j57", "_j56"))
#df.to_csv('j57_and_j56_eqtls.tsv', sep='\t')
df2 %>% head(2)

In [ ]:
title = paste0("eQTLs for DRD2 junctions 5-7 and 5-6 in GTEx accumbens\n(n=",dim(df)[1]," SNPs)")

sca = df2 %>% ggscatter(x="pval_nominal_j57", y="pval_nominal_j56", alpha=0.4, 
                 ggtheme=theme_pubclean(base_size=20), 
                 xlab="nominal p-value: junction 5-7",
                 ylab="nominal p-value: junction 5-6") +
    yscale("log10", .format=TRUE) + xscale("log10", .format=TRUE) +
    font("xy.title", face="bold") +
    ggtitle(title) + theme(plot.title = element_text(hjust = 0.5, face="bold"))
print(sca)
save_ggplots("eQTL_scatterplot_j57_j56_GTEx_accumbens", sca, 12, 10)

## Session Info

In [ ]:
Sys.time()
proc.time()
options(width = 120)
sessioninfo::session_info()